In [1]:
import os

In [2]:
os.environ["NEEDLE_API_KEY"] = ""


In [3]:
os.environ["OPENAI_API_KEY"] = ""

## Create a Collection

In [4]:
from needle_haystack import NeedleDocumentStore
from haystack.utils import Secret

# If not provided a new collection is generated, otherwise an existing collection is used.
# You can see your collections at https://needle-ai.com/dashboard/collections/
document_store = NeedleDocumentStore(api_key=Secret.from_env_var("NEEDLE_API_KEY"), name="TechRadarCollection", collection_id='clt_01J5J8HWYVWNSAP21A4WG6E332')

# Add files to your NeedleDocumentStore.
file_urls = {
    "tech-radar-30.pdf": "https://www.thoughtworks.com/content/dam/thoughtworks/documents/radar/2024/04/tr_technology_radar_vol_30_en.pdf"
}
document_store.write_documents(file_urls=file_urls)


In [14]:
from haystack import Pipeline
from haystack.utils import Secret
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders import PromptBuilder  # Correct import for PromptBuilder
from needle_haystack import NeedleRetriever

retriever = NeedleRetriever(document_store=document_store)

# Set up a prompt template for the PromptBuilder
prompt_template = """
Given the following retrieved documents, generate a concise and informative answer to the query:

Query: {{query}}
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Answer:
"""

# Initialize the PromptBuilder
prompt_builder = PromptBuilder(template=prompt_template)

# Initialize the OpenAIGenerator with the API key
llm = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"))

# Build the pipeline
basic_rag_pipeline = Pipeline()
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", llm)

# Connect the components
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

# Define the query
question = "What techniques moved to adopt?"

# Run the full pipeline
result = basic_rag_pipeline.run({
    "retriever": {"text": question},
    "prompt_builder": {"query": question}
})

# Print the final result generated by the LLM
print("Final generated result:")

print(result['llm']['replies'][0])

Final generated result:
To move towards adoption, the techniques recommended include Conan, Kaniko, and Karpenter. These techniques can help improve development processes and efficiency in software engineering.
